In [4]:
# import PyPDF2
import fitz
# import copy
import os
import re

In [5]:
file = "en_2022-2024 WAG COP.pdf"

In [79]:
def writeImage(image_path, image_bytes, count=1):
    # string = "page80_element_2.405_1.png"
    if count == 1:
        image_path = image_path.replace(".png", f"_{count}.png")
    else:
        image_path = re.sub(r"(\d?).png", count+".png", image_path)
    if os.path.exists(image_path):
        writeImage(image_path, image_bytes, count+1)
    else:
        with open(image_path, "wb") as image_file:
            image_file.write(image_bytes)

def getPageLayout(page):
    bbox = page.bleedbox
    if bbox.x1-bbox.y1 < 0:
        return "portrait"
    else:
        return "landscape"
    
def extract_text_and_images(pdf_path, output_folder, target_page, regex_pattern):
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Open the PDF file
    pdf_document = fitz.open(pdf_path)
    
    # Check if the target page is within the range of pages
    if target_page < 0 or target_page >= len(pdf_document):
        print("Error: Invalid target page number.")
        return
    
    page = pdf_document.load_page(target_page)
    page_layout = getPageLayout(page)

    # Extract text from the page
    page_text = page.get_text()
    # print(page_text)
    found_text = re.findall(regex_pattern, page_text)
    print("FOUND", found_text)
    # Extract images and their positions
    images = page.get_images(full=True)
    
    # Loop through images on the page
    for img_info in images:
        xref = img_info[0]
        base_image = pdf_document.extract_image(xref)
        image_bytes = base_image["image"]

        # Get the text closest and above the image
        bbox_image = page.get_image_bbox(img_info[7])
        # print("bbox img", bbox_image)

        if (bbox_image.bottom_right - bbox_image.top_left)[0] < 75:
            # assumed to be a 
            print("assuming", img_info[7], "is a judge symbol")
            continue


        found = ""
    
        optionA = []
        closestx = 1e6
        for item in found_text:
            for field in page.search_for(item):
                if page_layout == "portrait":
                    text_y1 = field.x1
                    text_x1 = abs(field.y0 - page.bleedbox.y1)
                else:
                    text_y1 = field.y1
                    text_x1 = field.x1
                x = round(bbox_image.y0 - round(text_y1))
                # print(field)
                # print(bbox_image)
                if x < 0:
                    continue
                if x < 15:
                    # In this case the number is too close to the picture and it is assumed its a number from the points; e.g. "3.20 P." instead of element 3.20
                    continue
                if x == closestx:
                    optionA.append(item)
                    closestx = x
                elif x < closestx:
                    optionA = [item]
                    closestx = x
                # print(optionA)
                for option in optionA:
                    # print(field)
                    # print(bbox_image.x0, text_x1, bbox_image.x1)
                    if bbox_image.x0 < text_x1 < bbox_image.x1:
                        # print("hurray", item)
                        found = option
                        pass
            
        if found != "":
            image_path = os.path.join(output_folder, f"page{target_page}_element_{found}.png")
            
            writeImage(image_path, image_bytes)
        else:
            print("warning! could not process ", img_info[7])
    pdf_document.close()

# Example usage
pdf_file_path = file
output_folder = "../data/images/"
target_page = 104  # Specify the page number you want to extract text and images from
regex_pattern = re.compile(r"(\d+\.\d+)\s?\n")  # Specify your regex pattern here
extract_text_and_images(pdf_file_path, output_folder, target_page, regex_pattern)


FOUND ['6.109', '6.209', '6.309', '6.409', '6.509', '6.110', '6.210', '6.310', '6.410', '6.510', '6.610']
assuming Im879 is a judge symbol
assuming Im893 is a judge symbol
assuming Im882 is a judge symbol
assuming Im884 is a judge symbol
assuming Im892 is a judge symbol
assuming Im877 is a judge symbol
assuming Im888 is a judge symbol


TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [86]:
pdf_document = fitz.open(file)
page = pdf_document.load_page(page_id=104)
getPageLayout(page=page)

'landscape'

In [87]:
page_text = page.get_text()
# print(page_text)
found_text = re.findall(r"(\d+\.\d+)\s+\n+(?!P\.)", page_text)
print(found_text)

['6.109', '6.209', '6.309', '6.409', '6.509', '6.609', '6.110', '6.210', '6.310', '6.410', '6.510', '6.610']


In [88]:
page_text

'6.000 DISMOUNTS \nUB – Group 6 - 7 \nA \nB \nC \nD \nE \nF/G \n6.109 \n \n6.209 \nHip circle bwd (also clear) on \nLB or HB – hecht \n \n \n6.309 \nHip circle bwd (also clear) on LB or HB – \nhecht with 1/1 turn (360°) \n \n6.409 \nHip circle bwd (also clear) on LB or \nHB – hecht with salto bwd tucked  \n   (Mukhina) \n \n \n6.509 \nHip circle bwd (also clear) on LB \nor HB – hecht with ½ turn (180°) \nto salto fwd tucked \n \n \n6.609  \nHip circle bwd (also clear) on \nLB or HB – hecht with 1/1 turn \n(360°) to salto bwd tucked  \n   (Ma) \n \n \n \n \n6.110 \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n6.210 \nOn HB - salto fwd tucked \n \n \n6.310 \nOn HB – salto fwd piked or salto bwd \ntucked or piked  \n   (Delladio / Parolari) \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n6.410 \n \n6.510 \n \n \n6.610 \n \n \n \n'

In [89]:
n = page.get_images()[17]
page.get_image_bbox(n[7])


Rect(292.9999694824219, 224.451171875, 391.7850646972656, 293.2025451660156)

'landscape'

In [90]:
# left, up, right, bottom
# clip = fitz.Rect(28.19099998474121 - 50, 113.76000213623047 - 50, 145.79100036621094 + 50, 186.24000549316406)
clip = page.get_image_bbox(page.get_images()[17][7])
# clip = fitz.Rect(168.59100341796875, 103.19999694824219 - 50, 277.0710144042969, 162.72000122070312)
pix = page.get_pixmap(matrix=fitz.Matrix(2, 2), clip=clip)
pix.save("test.png")

In [105]:
optionA = []
closestx = 1e6
for item in found_text:
    fields = page.search_for(item)
    # print(fields)
    # continue
    for field in fields:
        left,up,right,bottom = field
        text_y1 = field.y1
        # text_x1 = field.x1
        text_x1= page.search_for(item)[0].x1

        # x = round(bbox_image.y0 - round(text_y1))
                
        x = round(clip.y0 - round(number=text_y1))
        print(item,    field)
        # print(x)
        # print(item,  page.bleedbox.x1 - left )
        # print(item, x, clip.y0, right)
        if x < 0:
            continue
        if x < 15:
            continue
        if x == closestx:
            optionA.append(item)
            closestx = x
        elif x < closestx:
            optionA = [item]
            closestx = x
print(optionA)
for i in optionA:
    # print(clip.x0, abs(x1_text- page.bleedbox.y1), clip.x1)
    if clip.x0 < abs(text_x1- page.bleedbox.y1) < clip.x1:
        print("hurray", i)
        pass
print(clip)              
optionA
    # print(round(left))

6.109 Rect(28.31999969482422, 57.42161560058594, 48.38102722167969, 66.40380096435547)
6.209 Rect(146.3000030517578, 57.42161560058594, 166.36102294921875, 66.40380096435547)
6.309 Rect(271.489990234375, 57.42161560058594, 291.551025390625, 66.40380096435547)
6.409 Rect(428.69000244140625, 57.42161560058594, 448.75103759765625, 66.40380096435547)
6.509 Rect(570.1900024414062, 57.42161560058594, 590.2510986328125, 66.40380096435547)
6.609 Rect(699.8200073242188, 57.42161560058594, 719.881103515625, 66.40380096435547)
6.110 Rect(28.31999969482422, 333.3515930175781, 48.38102722167969, 342.33380126953125)
6.210 Rect(146.3000030517578, 333.3515930175781, 166.36102294921875, 342.33380126953125)
6.310 Rect(271.489990234375, 333.3515930175781, 291.551025390625, 342.33380126953125)
6.410 Rect(428.69000244140625, 333.3515930175781, 448.75103759765625, 342.33380126953125)
6.510 Rect(570.1900024414062, 333.3515930175781, 590.2510986328125, 342.33380126953125)
6.610 Rect(699.8200073242188, 333.351

['6.109', '6.209', '6.309', '6.409', '6.509', '6.609']

In [60]:
page.get_textbox((358.2958984375, 129.9893798828125, 367.2110595703125, 147.74560546875))

'3.35 '

In [120]:
len(os.listdir("../data/images/vault"))

87